In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# This software may be used and distributed according to the terms of the GNU General Public License version 3.

import os
import torch
import fire
import time
import json

from pathlib import Path


os.environ["BITSANDBYTES_NOWELCOME"] = "1"
from llama import ModelArgs, Transformer, Tokenizer, LLaMA, default_quantize

def load(
    ckpt_dir: str,
    tokenizer_path: str,
    max_seq_len: int,
    max_batch_size: int,
    quantize: bool,
) -> LLaMA:
    start_time = time.time()
    checkpoints = sorted(Path(ckpt_dir).glob("*.pth"))

    with open(Path(ckpt_dir) / "params.json", "r") as f:
        params = json.loads(f.read())

    model_args: ModelArgs = ModelArgs(
        max_seq_len=max_seq_len, max_batch_size=max_batch_size, **params
    )
    tokenizer = Tokenizer(model_path=tokenizer_path)
    model_args.vocab_size = tokenizer.n_words

    torch.set_default_tensor_type(torch.HalfTensor)
    print("Allocating transformer on host")
    ctx_tok = default_quantize.set(quantize)
    model = Transformer(model_args)
    default_quantize.reset(ctx_tok)
    key_to_dim = {
        "w1": 0,
        "w2": -1,
        "w3": 0,
        "wo": -1,
        "wq": 0,
        "wk": 0,
        "wv": 0,
        "output": 0,
        "tok_embeddings": -1,
        "ffn_norm": None,
        "attention_norm": None,
        "norm": None,
        "rope": None,
    }

    # ?
    torch.set_default_tensor_type(torch.FloatTensor)

    # for i, ckpt in enumerate(checkpoints):
    #     print(f"Loading checkpoint {i}")
    #     checkpoint = torch.load(ckpt, map_location="cpu")
    #     for parameter_name, parameter in model.named_parameters():
    #         short_name = parameter_name.split(".")[-2]
    #         if key_to_dim[short_name] is None and i == 0:
    #             parameter.data = checkpoint[parameter_name]
    #         elif key_to_dim[short_name] == 0:
    #             size = checkpoint[parameter_name].size(0)
    #             parameter.data[size * i : size * (i + 1), :] = checkpoint[
    #                 parameter_name
    #             ]
    #         elif key_to_dim[short_name] == -1:
    #             size = checkpoint[parameter_name].size(-1)
    #             parameter.data[:, size * i : size * (i + 1)] = checkpoint[
    #                 parameter_name
    #             ]
    #         del checkpoint[parameter_name]
    #     del checkpoint
    return model   
    # model.cuda()

    # generator = LLaMA(model, tokenizer)
    # print(
    #     f"Loaded in {time.time() - start_time:.2f} seconds with {torch.cuda.max_memory_allocated() / 1024 ** 3:.2f} GiB"
    # )
    # return generator



In [2]:
temperature = 0.8
top_p = 0.95
repetition_penalty_range = 1024
repetition_penalty_slope = 0
repetition_penalty = 1.15
max_seq_len = 512
max_batch_size = 32
ckpt_dir = "/mnt/8b4bef76-4da0-408d-bbc6-970c03cccc81/1_data_backup/llama2/github/llama-2-13b"
tokenizer_path = "/home/usr1/llama/llama-main/tokenizer.model"
use_int8 = False
generator = load(ckpt_dir, tokenizer_path, max_seq_len, max_batch_size, use_int8)

prompts = [
    # For these prompts, the expected answer is the natural continuation of the prompt
    """Welcome.
The following conversation took place at Harvard University.
Former Treasurer Secretary Larry Summers invited Ray Dalio, the founder, chairman and
co-CIO of Bridgewater Associates, the world's largest hedge fund, to discuss Dalio's unique
views on economics.

Dalio:""",
]
results = generator.generate(
    prompts,
    max_gen_len=1024,
    temperature=temperature,
    top_p=top_p,
    repetition_penalty_range=repetition_penalty_range,
    repetition_penalty_slope=repetition_penalty_slope,
    repetition_penalty=repetition_penalty,
)

for result in results:
    print(result)
    print("\n==================================\n")

Allocating transformer on host


AttributeError: 'Transformer' object has no attribute 'generate'

In [2]:
temperature = 0.8
top_p = 0.95
repetition_penalty_range = 1024
repetition_penalty_slope = 0
repetition_penalty = 1.15
max_seq_len = 512
max_batch_size = 32
ckpt_dir = "/mnt/8b4bef76-4da0-408d-bbc6-970c03cccc81/1_data_backup/llama2/github/llama-2-13b"
tokenizer_path = "/home/usr1/llama/llama-main/tokenizer.model"
use_int8 = True
generator = load(ckpt_dir, tokenizer_path, max_seq_len, max_batch_size, use_int8)

prompts = [
    # For these prompts, the expected answer is the natural continuation of the prompt
    """Welcome.
The following conversation took place at Harvard University.
Former Treasurer Secretary Larry Summers invited Ray Dalio, the founder, chairman and
co-CIO of Bridgewater Associates, the world's largest hedge fund, to discuss Dalio's unique
views on economics.

Dalio:""",
]

Allocating transformer on host


In [3]:
generator.layers[0].attention.wq.weight

Parameter containing:
Parameter(Int8Params([[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float16))

In [ ]:
Parameter containing:
tensor([[ 0.0002, -0.0022, -0.0065,  ...,  0.0009,  0.0042,  0.0032],
        [-0.0004,  0.0054,  0.0111,  ..., -0.0017,  0.0069,  0.0015],
        [ 0.0007,  0.0016,  0.0051,  ...,  0.0027,  0.0038,  0.0018],
        ...,
        [ 0.0036,  0.0068,  0.0061,  ...,  0.0051, -0.0029,  0.0004],
        [-0.0051, -0.0066, -0.0018,  ..., -0.0067,  0.0030, -0.0010],
        [-0.0029, -0.0052, -0.0066,  ..., -0.0046,  0.0033,  0.0001]],
       dtype=torch.float16, requires_grad=True)

In [ ]:
Parameter containing:
Parameter(Int8Params([[ 0.0002, -0.0022, -0.0065,  ...,  0.0009,  0.0042,  0.0032],
            [-0.0004,  0.0054,  0.0111,  ..., -0.0017,  0.0069,  0.0015],
            [ 0.0007,  0.0016,  0.0051,  ...,  0.0027,  0.0038,  0.0018],
            ...,
            [ 0.0036,  0.0068,  0.0061,  ...,  0.0051, -0.0029,  0.0004],
            [-0.0051, -0.0066, -0.0018,  ..., -0.0067,  0.0030, -0.0010],
            [-0.0029, -0.0052, -0.0066,  ..., -0.0046,  0.0033,  0.0001]],
           dtype=torch.float16))

In [ ]:
torch.save(generator.state_dict(), "13b_int8.pth")